In [10]:
pip install git+https://github.com/LIAAD/yake

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-w230neef
  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake /tmp/pip-req-build-w230neef
  Resolved https://github.com/LIAAD/yake to commit 374fc1c1c19eb080d5b6115cbb8d4a4324392e54
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 454.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 2.9 MB/s eta 0:00:0000:0100:01
Using legacy 'setup.py install' for yake, since package 'wheel' is not installed.
  Running setup.py install for yake ... done
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install rake-nltk

In [86]:
!python3 -m pip install pytextrank
!python3 -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 7.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 7.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 6.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.3/914.3 KB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 KB 4.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 KB 3.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.9/370

In [2]:
import os, re
from nltk.tokenize import word_tokenize

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from rake_nltk import Rake

import yake

import spacy, pytextrank

[nltk_data] Downloading package stopwords to /home/sergi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sergi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
class Extractor:
    def __init__(self, language, max_tokens):
        self.language = language
        self.max_tokens = max_tokens
        pass

    def extract_terms(self, text):
        raise NotImplementedError("extract_terms method must be implemented in subclass")

    def extract_terms_with_span(self, text):
        terms = self.extract_terms(text)
        terms_with_span = self.find_term_span(text, terms)
        return terms_with_span

    def extract_terms_without_overlaps(self, text):
        terms_with_span = self.extract_terms_with_span(text)
        terms_without_overlaps = self.rmv_overlaps(terms_with_span)
        return terms_without_overlaps

    def postprocess_terms(self, terms):
        pass

    @staticmethod
    def find_term_span(text, terms):
        spans = []
        for t in terms:
          term = re.escape(t)
          patron = r'\b' + term + r'\b'
          coincidencias = re.finditer(patron, text, re.IGNORECASE)
          span = [[coincidencia.start(), coincidencia.end()-1] for coincidencia in coincidencias]
          spans.append((t,span))
        return spans

    @staticmethod
    def rmv_overlaps(keywords):
      ent = [kw[0] for kw in keywords]
      pos = [kw[1] for kw in keywords]
      updated_keywords = []
      n = 0
      for i in range(len(ent)):
        for j in range(len(pos[i])):
          overlap = False
          for k in range(len(ent)):
            for l in range(len(pos[k])):
              if ((pos[i][j][0] >= pos[k][l][0]) and (pos[i][j][1] <= pos[k][l][1]) and i!=k):
                overlap = True
          if (not overlap):
            updated_keywords.append((ent[i],pos[i][j][0],pos[i][j][1]))
      return updated_keywords

In [4]:
class RakeExtractor(Extractor):
    def __init__(self, language, max_tokens):
        super().__init__(language, max_tokens)
        self.stopwords = nltk.corpus.stopwords.words(language)
        self.extractor = Rake(stopwords=self.stopwords, language=language)

    def extract_terms(self, text):
        self.extractor.extract_keywords_from_text(text)
        terms = self.extractor.get_ranked_phrases()
        terms = [kw for kw in terms if (len(word_tokenize(kw)) <= self.max_tokens)]
        return terms

In [5]:
class YakeExtractor(Extractor):
    def __init__(self, language, max_tokens):
        super().__init__(language, max_tokens)
        if language=='spanish':
            self.extractor = yake.KeywordExtractor() #aqui habria que poner top=70 por ejemplo
        else:
            raise ValueError("Expected spanish language. Other languages not recognised")

    def extract_terms(self, text):
        keywords = self.extractor.extract_keywords(text)
        terms = [kw for kw, score in keywords if (len(word_tokenize(kw)) <= self.max_tokens)]
        return terms

In [6]:
class TextRankExtractor(Extractor):
    def __init__(self, language, max_tokens):
        super().__init__(language, max_tokens)
        if language=='spanish':
            self.extractor = spacy.load("es_core_news_sm")
            self.extractor.add_pipe("textrank")
        else:
            raise ValueError("Expected spanish language. Other languages not recognised")

    def extract_terms(self, text):
        doc = self.extractor(text)
        terms = []
        for phrase in doc._.phrases:
          if (len(word_tokenize(phrase.text)) <= self.max_tokens):
            terms.append(phrase.text)
        return terms

In [15]:
class TermExtractor:
    def __init__(self, extraction_methods="rake", language="spanish", max_tokens=3): #maybe fer que max_tokens s hagi de posar mes tard??
        self.extraction_methods = extraction_methods
        self.extractors = self.initialize_keyword_extractors(language, max_tokens)

    def __call__(self, text):
        terms = self.extract_terms(text)
        return terms
        
    def initialize_keyword_extractors(self, language, max_tokens):
        keyword_extractors = {} #esto esta hecho para mas de un extractor a la vez???
        
        if 'rake' in self.extraction_methods:
            keyword_extractors["rake"] = RakeExtractor(language, max_tokens)
        
        if 'yake' in self.extraction_methods:
            keyword_extractors["yake"] = YakeExtractor(language, max_tokens)
        
        if 'textrank' in self.extraction_methods:
            keyword_extractors["textrank"] = TextRankExtractor(language, max_tokens)

        if not keyword_extractors:
            raise ValueError("No extraction method called {}".format(self.extraction_methods))
        
        return keyword_extractors

    def extract_terms(self, text):
        if (len(self.extractors) == 1):
            terms = self.extractors[self.extraction_methods].extract_terms_without_overlaps(text)
        else:
            all_terms = []
            for extractor in self.extractors.values():
                all_terms.extend(extractor.extract_terms_with_span(text))
            terms = self.extractors[self.extraction_methods[0]].rmv_overlaps(all_terms)
        return terms
        

In [24]:
extractor = TermExtractor(extraction_methods="textrank")

In [25]:
extractor("La importancia vital de lavarse con jabón las manos antes de operar es esencial")

[('jabón', 36, 40), ('La importancia vital', 0, 19), ('las manos', 42, 50)]

In [ ]:
#estaria be fer el post processing abans de rmv_overlaps perq aixi les que passen a ser insignificants despres del postprocessing les eliminem

In [10]:
#Test de extracción completa directa
text = "Un varón de 32 años acude al Servicio de Urgencias por disminución reciente de visión en OD coincidiendo con la aparición de una lesión parduzca en dicho ojo. Entre los antecedentes oftalmológicos destaca un traumatismo penetrante en OD tres años antes que fue suturado en nuestro centro. A la exploración presenta una agudeza visual de 0,1 que mejora a 0,5 con estenopéico."

extractor1 = TermExtractor(extraction_methods="textrank", language="spanish", max_tokens=5)
print("Usando TextRank y un máximo de 5 tokens: \n",extractor1(text))

extractor2 = TermExtractor(extraction_methods=["rake", "yake"], language="spanish", max_tokens=2)
print("\nUsando Rake y Yake juntos y un máximo de 2 tokens: \n",extractor2(text))

extractor3 = TermExtractor(extraction_methods=["rake", "yake", "textrank"], language="spanish", max_tokens=3)
print("\nUsando Rake, Yake y TextRank juntos y un máximo de 3 tokens: \n",extractor3(text))

Usando TextRank y un máximo de 5 tokens: 
 [('dicho ojo', 148, 156), ('disminución reciente', 55, 74), ('OD', 89, 90), ('OD', 234, 235), ('estenopéico', 362, 372), ('visión', 79, 84), ('una lesión parduzca', 125, 143), ('Servicio de Urgencias', 29, 49), ('un traumatismo penetrante', 205, 229), ('nuestro centro', 273, 286), ('una agudeza visual', 315, 332), ('la aparición', 109, 120), ('32 años', 12, 18), ('tres años', 237, 245), ('Entre los antecedentes oftalmológicos', 159, 195), ('Un varón', 0, 7), ('A la exploración', 289, 304), ('que', 253, 255), ('que', 341, 343)]

Usando Rake y Yake juntos y un máximo de 2 tokens: 
 [('od coincidiendo', 89, 103), ('traumatismo penetrante', 208, 229), ('exploración presenta', 294, 313), ('agudeza visual', 319, 332), ('visión', 79, 84), ('varón', 3, 7), ('suturado', 261, 268), ('mejora', 345, 350), ('estenopéico', 362, 372), ('centro', 281, 286), ('aparición', 112, 120), ('5', 356, 356), ('1', 339, 339), ('Urgencias por', 41, 53), ('por disminución

In [14]:
#Test de extracción por pasos: vemos que da lo mismo que el de antes
extractor = TermExtractor(extraction_methods="textrank", language="spanish", max_tokens=3)
terms = extractor.extractors["textrank"].extract_terms(text)
terms_with_span = extractor.extractors["textrank"].find_term_span(text,terms)
terms_without_overlaps = extractor.extractors["textrank"].rmv_overlaps(terms_with_span)
print(terms_without_overlaps)

[('dicho ojo', 148, 156), ('disminución reciente', 55, 74), ('OD', 89, 90), ('OD', 234, 235), ('estenopéico', 362, 372), ('visión', 79, 84), ('una lesión parduzca', 125, 143), ('Servicio de Urgencias', 29, 49), ('un traumatismo penetrante', 205, 229), ('nuestro centro', 273, 286), ('una agudeza visual', 315, 332), ('la aparición', 109, 120), ('32 años', 12, 18), ('tres años', 237, 245), ('los antecedentes oftalmológicos', 165, 195), ('Un varón', 0, 7), ('A la exploración', 289, 304), ('que', 253, 255), ('que', 341, 343)]


In [115]:
extractor.extractors["rake"].stopwords

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [17]:
extractor.extractors["rake"].extract_terms("La importancia vital de lavarse con jabón las manos antes de operar es indispensable")

['importancia vital', 'operar', 'manos', 'lavarse', 'jabón', 'indispensable']

In [ ]:
extractor_terminos = TermExtractor(sadasdasdsa)

In [ ]:
doc1_terms = extractor_terminos("textoasdasdasdsa")

In [ ]:
doc1_terms.zº

In [60]:
import sys, os, re
general_path = os.getcwd().split("BioTermCategorizer")[0]+"BioTermCategorizer/"
sys.path.append(general_path+'biotermcategorizer/')


In [36]:
from TermExtractor import TermExtractor

In [37]:
extractor = TermExtractor(extraction_methods=["textrank","rake"])

In [39]:
extractor("texto de prueba")

{'rake': 'OBJETO YAKE EXTRACTOR', 'textrank': 'OBJETO TEXTRANK EXTRACTOR'}
texto de prueba


In [54]:
rake_extractor = RakeExtractor(stopwords=["a","b","c"], language = "spanish")

In [55]:
rake_extractor.extract_terms("hola que tal")

NotImplementedError: extract_terms method must be implemented in subclass